In [1]:
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import json
import pickle
import re
from nltk.tokenize import word_tokenize
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
# noinspection PyStubPackagesCompatibility
pd.set_option('display.max_columns', None)

In [3]:
file_data = json.load(open(os.path.expanduser("../../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

lagt_metadata_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/10pGulpiwzjUozVEVstKBwtftyDisSY9h7-kl82TVs0A/edit?usp=sharing")

In [4]:
greek_data_dir = "/srv/data/greek/"
os.listdir(greek_data_dir)

['oga_graphannis',
 'oga_sentences_2025-08',
 'oga_sentences',
 'glaux_sentences',
 'grela_v0-2.duckdb',
 'OGA',
 'glaux',
 'exprecce_sentences_2025-08',
 'exprecce',
 'grela.duckdb.tmp',
 'exprecce_sentences_2025-10',
 'grela.duckdb',
 'glaux_sentences_2025-08',
 'LAGT',
 'OGA_0-2-0.zip',
 'grela_v0-2.duckdb.wal',
 'opera_graeca_adnotata_v0.2.0',
 'exprecce_sentences']

In [5]:
oga_sentences_dir = greek_data_dir + "oga_sentences_2025-08/"
glaux_sentences_dir = greek_data_dir + "glaux_sentences_2025-08/"
exprecce_sentences_dir = greek_data_dir + "exprecce_sentences_2025-10/"
sentences_data = {
    "oga": {"dir" : oga_sentences_dir},
    "glaux": {"dir" : glaux_sentences_dir},
    "exprecce": {"dir", exprecce_sentences_dir}
}
paths = [oga_sentences_dir, glaux_sentences_dir, exprecce_sentences_dir]

In [7]:
# test
# from 5th file in each source corpus
# print 5th sentence
for path in paths:
    fn = os.listdir(path)[5]
    with open(os.path.join(path, fn), "rb") as f:
        sents_data = pickle.load(f)
    print(sents_data[5])

('tlg0026.tlg004', 5, 'ταῦτα δὲ ἐπείθοντο κατὰ μῆνιν Ἀπόλλωνος αὐτοῖς συμβεβηκέναι, ταφέντος ἐν τῇ νήσῳ τινὸς τῶν ἐπι.', [('ταῦτα', 'οὗτος', 'PRON', {'oga_cts': '1_2', 'oga_tid': 't_88'}, 0, 5), ('δὲ', 'δέ', 'ADV', {'oga_cts': '1_2', 'oga_tid': 't_89'}, 6, 8), ('ἐπείθοντο', 'πείθω', 'VERB', {'oga_cts': '1_2', 'oga_tid': 't_90'}, 9, 18), ('κατὰ', 'κατά', 'ADP', {'oga_cts': '1_2', 'oga_tid': 't_91'}, 19, 23), ('μῆνιν', 'μῆνις', 'NOUN', {'oga_cts': '1_2', 'oga_tid': 't_92'}, 24, 29), ('Ἀπόλλωνος', 'ἀπόλλων', 'NOUN', {'oga_cts': '1_2', 'oga_tid': 't_93'}, 30, 39), ('αὐτοῖς', 'αὐτός', 'PRON', {'oga_cts': '1_2', 'oga_tid': 't_94'}, 40, 46), ('συμβεβηκέναι', 'συμβαίνω', 'VERB', {'oga_cts': '1_2', 'oga_tid': 't_95'}, 47, 59), (',', ',', 'PUNCT', {'oga_cts': '1_2', 'oga_tid': 't_96'}, 59, 60), ('ταφέντος', 'θάπτω', 'VERB', {'oga_cts': '1_2', 'oga_tid': 't_97'}, 61, 69), ('ἐν', 'ἐν', 'ADP', {'oga_cts': '1_2', 'oga_tid': 't_98'}, 70, 72), ('τῇ', 'ὁ', 'DET', {'oga_cts': '1_2', 'oga_tid': 't_99'}, 

## LAGT v3.0

In [61]:
LAGT = pd.read_parquet(greek_data_dir + "LAGT/LAGT_v3-0.parquet")

In [62]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience,lemmatacount
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,None,101.0,200.0,None,[],christian,34
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,4-3 B.C.,-400.0,-201.0,False,[Bucolici],pagan,61
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,5 B.C.,-500.0,-401.0,False,[Tragici],pagan,10277
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,1488
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,125


In [63]:
LAGT.groupby("source").size()

source
1Kgr       949
perseus    761
dtype: int64

In [64]:
LAGT.groupby("lemmata_source").size()

lemmata_source
agdt         25
glaux       835
gorman        2
grecy       762
lxxmorph     55
morphgnt     27
pedalion      4
dtype: int64

In [65]:
LAGT["tokencount"] = LAGT["string"].apply(lambda x: len(word_tokenize(x)))

In [66]:
LAGT3_metadata = LAGT[['doc_id', 'lemmata_source', 'tokencount']].copy()
for col in LAGT3_metadata.columns:
    if col !="doc_id":
        LAGT3_metadata.rename(columns={col : "lagt3_" + col}, inplace=True)

## LAGT v4.1

In [12]:
LAGT = pd.read_parquet(greek_data_dir + "LAGT/LAGT_v4-1.parquet")

In [13]:
LAGT.columns

Index(['author_id', 'doc_id', 'filename', 'author', 'title', 'sentences',
       'lemmatized_sentences', 'source', 'lemmata_source', 'not_before',
       'not_after', 'tlg_epithet', 'genre', 'provenience', 'wordcount',
       'lemmatacount'],
      dtype='object')

In [14]:
LAGT["string"] = LAGT["sentences"].apply(lambda x: " ".join(x))

In [15]:
LAGT["tokencount"] = LAGT["string"].apply(lambda x: len(word_tokenize(x)))

In [16]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,sentences,lemmatized_sentences,source,lemmata_source,not_before,not_after,tlg_epithet,genre,provenience,wordcount,lemmatacount,string,tokencount
0,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"[, , ., . . . . ., — πρός ἥν ( ,, ), ὁ Πινυτός...","[[], [], [], [], [πινυτός, ἀντιγράφω], [θαυμάζ...",glaux1,glaux1,101.0,200.0,[],[],christian,109,34,", , . . . . . . — πρός ἥν ( , ) ὁ Πινυτός ἀντι...",108
1,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,[Οὐδενός εὐνάτειρα Μακροπτολέμοιο δέ μάτηρ μαί...,"[[εὐνητήρ, μακροπτολέμον, μήτηρ, μαῖα, ἀντιπέτ...",glaux1,glaux1,-400.0,-201.0,[Bucolici],[],pagan,95,59,Οὐδενός εὐνάτειρα Μακροπτολέμοιο δέ μάτηρ μαία...,94
2,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,[ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσ...,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",glaux1,glaux1,-500.0,-401.0,[Tragici],[],pagan,21516,10315,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,21729
3,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,[Οἴκοι τά Μιλήσια: ἐπί τῶν ὅποι μή προςήκει τή...,"[[μιλήσιος, πργοςήκω, τρυφή, ἐπιδείκνυμι], [ἀρ...",glaux1,glaux1,1.0,200.0,"[Biographi, Philosophici/-ae]",[],pagan,3206,1496,Οἴκοι τά Μιλήσια: ἐπί τῶν ὅποι μή προςήκει τήν...,3178
4,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,"[Κατά πετρῶν σπείρεις., Πλίνθον πλύνεις., Δικτ...","[[πέτρα, σπείρω], [πλίνθος, πλύνω], [δίκτυον, ...",glaux1,glaux1,1.0,200.0,"[Biographi, Philosophici/-ae]",[],pagan,195,125,Κατά πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,194


In [25]:
LAGT41_metadata = LAGT[['doc_id', 'author', 'title', 'not_before',
       'not_after', 'tlg_epithet', 'source', 'genre', 'provenience', "tokencount"]].copy()
for col in LAGT41_metadata.columns:
    if col !="doc_id":
        LAGT41_metadata.rename(columns={col : "lagt4-1_" + col}, inplace=True)
LAGT41_metadata.head(5)

,doc_id,lagt4-1_author,lagt4-1_title,lagt4-1_not_before,lagt4-1_not_after,lagt4-1_tlg_epithet,lagt4-1_source,lagt4-1_genre,lagt4-1_provenience,lagt4-1_tokencount
0,ogl0001.ogl001,Pinytus,De Epistola Pinyti ad Dionysium,101.0,200.0,[],glaux1,[],christian,108
1,tlg0005.tlg003,Theocritus,Syrinx,-400.0,-201.0,[Bucolici],glaux1,[],pagan,94
2,tlg0006.tlg020,Euripides,Fragmenta,-500.0,-401.0,[Tragici],glaux1,[],pagan,21729
3,tlg0007.tlg146,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,1.0,200.0,"[Biographi, Philosophici/-ae]",glaux1,[],pagan,3178
4,tlg0007.tlg147,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,1.0,200.0,"[Biographi, Philosophici/-ae]",glaux1,[],pagan,194


## Glaux metadata

In [18]:
glaux_metadata = pd.read_csv("/srv/data/greek/glaux/metadata.txt", sep="\t")
glaux_metadata.head(5)

,GLAUX_TEXT_ID,TLG,STARTDATE,ENDDATE,AUTHOR_STANDARD,TITLE_STANDARD,GENRE_STANDARD,DIALECT,SOURCE,SOURCE_LICENSE,SOURCE_FORMAT,TOKENS,TM_TEXT
0,1,0012-001,-800,-701,Homerus,Ilias,Epic poetry,Ionic/Epic,Perseus,CC BY-SA 4.0,XML,129604,511
1,2,0012-002,-800,-701,Homerus,Odyssea,Epic poetry,Ionic/Epic,Perseus,CC BY-SA 4.0,XML,104364,512
2,3,0012-003,-800,-701,Homerus,Epigrammata,Lyric poetry,Ionic/Epic,Perseus,CC BY-SA 4.0,XML,26,12612
3,4,1351-001,-800,-701,Epigoni,Epigoni,Epic poetry,Ionic/Epic,https://sententiaeantiquae.com,NaN,TXT,60,13805 / 15768
4,5,1547-001,-800,-701,Oedipodea,Oedipodea,Epic poetry,Ionic/Epic,https://sententiaeantiquae.com,NaN,TXT,15,12913


In [19]:
glaux_tlg = "0012-001"
groups = re.search(r"(\d{4})\-(\d{3})", glaux_tlg).groups()

In [20]:
def tlg_parsing(glaux_tlg):
    groups = re.search(r"(\d{4})\-(\d{3})", glaux_tlg).groups()
    tlg_id = "tlg"+ groups[0] + ".tlg" + groups[1]
    return tlg_id
glaux_metadata["tlg_id"] = glaux_metadata["TLG"].apply(lambda x: tlg_parsing(x))

In [21]:
for col in glaux_metadata.columns:
    glaux_metadata.rename(columns={col : "glaux_" + col}, inplace=True)
glaux_metadata.head(5)

,glaux_GLAUX_TEXT_ID,glaux_TLG,glaux_STARTDATE,glaux_ENDDATE,glaux_AUTHOR_STANDARD,glaux_TITLE_STANDARD,glaux_GENRE_STANDARD,glaux_DIALECT,glaux_SOURCE,glaux_SOURCE_LICENSE,glaux_SOURCE_FORMAT,glaux_TOKENS,glaux_TM_TEXT,glaux_tlg_id
0,1,0012-001,-800,-701,Homerus,Ilias,Epic poetry,Ionic/Epic,Perseus,CC BY-SA 4.0,XML,129604,511,tlg0012.tlg001
1,2,0012-002,-800,-701,Homerus,Odyssea,Epic poetry,Ionic/Epic,Perseus,CC BY-SA 4.0,XML,104364,512,tlg0012.tlg002
2,3,0012-003,-800,-701,Homerus,Epigrammata,Lyric poetry,Ionic/Epic,Perseus,CC BY-SA 4.0,XML,26,12612,tlg0012.tlg003
3,4,1351-001,-800,-701,Epigoni,Epigoni,Epic poetry,Ionic/Epic,https://sententiaeantiquae.com,NaN,TXT,60,13805 / 15768,tlg1351.tlg001
4,5,1547-001,-800,-701,Oedipodea,Oedipodea,Epic poetry,Ionic/Epic,https://sententiaeantiquae.com,NaN,TXT,15,12913,tlg1547.tlg001


In [22]:
glaux_metadata["doc_id"] = glaux_metadata["glaux_tlg_id"]

## OGA metadata

In [26]:
# load the following file with beautiful soup
filepath = "/srv/data/greek/opera_graeca_adnotata_v0.2.0/work_chronology/texts/chronology_greek_works_plus_date_label.xml"
with open(filepath, 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'xml')

In [27]:
# Extract records
records = []
for record in soup.find_all("record"):
    record_data = {}
    for field in record.find_all():
        record_data[field.name] = field.text.strip()  # Use tag name as key, text content as value
    records.append(record_data)

# Convert to DataFrame
oga_metadata = pd.DataFrame(records)

In [28]:
oga_metadata.head(5)

,id,urn_cts,title_labels,title_from_print_edition,print_edition,author,estimated_work_date,is_temporary_work_date,date_source,date_source_link,comment_on_chronology,formatted_work_date,date_label
0,1,ogl0001.ogl001,De Epistola Pinyti ad Dionysium,De Epistola Pinyti ad Dionysium,"Pinytus, Saint, Bishop of Knossos. Reliquiae S...","Pinytus, Saint, Bishop of Knossos",2nd Century AD (101-200),No,"Kritiko Agiologio, Holy Archibisophry of Crete",http://www.iak.gr/gr/ekklisia-kritis/kritiko_a...,Church of Crete Saints' Days Catalogue. The da...,+0101-01/+0200-12,p2_2
1,2,stoa0033a.tlg028,De mundo,De mundo,"pseudo-Aristotle, De mundo, Aristotelis Opera,...",Pseudo-Aristotle,250 BC-50BC,No,Pseudo-Aristotle: De Mundo (On the Cosmos). Ca...,https://www.cambridge.org/core/books/pseudoari...,Written between the middle of the 3rd and the ...,—0249-01/—0049-12,m3_2/m2_1/m2_2/m1_1
2,3,stoa0033a.tlg043,De spiritu,De spiritu,"pseudo-Aristotle, De spiritu, Aristotelis Oper...",Pseudo-Aristotle,275 BC-250 BC,No,Pseud-Aristotelian De Spiritu: A New Case agai...,,Follows Jaeger's proposed chronology.,—0274-01/—0249-12,m3_1
3,4,stoa0121.stoa001,Breviarium historiae romanae,Breviarium historiae romanae,Eutropius. Breviarium historiae romanae. Droys...,Eutropius,364 AD-378 AD,No,"Eutropius, Livius.org, 2020",https://www.livius.org/articles/person/eutropius/,Proposes an exact date of 369 AD.,+0364-01/+0378-12,p4_2
4,5,stoa0146d.stoa001,Acta Archelai,Acta Archelai,"Hegemonius. Acta Archelai. Beeson, Charles Hen...",Hegemonius,280 AD - 350 AD,No,"Archelaos, Wikisource",https://de.wikisource.org/wiki/RE:Archelaos_40,Based on the fact that he was bishop around 28...,+0280-01/+0350-12,p4_1


In [29]:
len(oga_metadata)

1911

In [30]:
oga_metadata["formatted_work_date"].tolist()[:10]

['+0101-01/+0200-12',
 '—0249-01/—0049-12',
 '—0274-01/—0249-12',
 '+0364-01/+0378-12',
 '+0280-01/+0350-12',
 '—0299-01/—0200-12',
 '—0430-01/—0410-12',
 '+0222-01/+0235-12',
 '—0299-01/—0259-12',
 '—0299-01/—0259-12']

In [31]:
def parse_dates(formatted_date):
    # Split the string into not_before and not_after parts
    not_before, not_after = formatted_date.split("/")

    # Convert not_before and not_after consistently
    not_before = not_before.replace("+", "").replace("—", "-")
    if not_before[0]=="-":
        not_before = int(not_before[:5])
    else:
        not_before = int(not_before[:4])

    # Replace em dash and +, handle full string
    not_after = not_after.replace("+", "").replace("—", "-")
    if not_after[0]=="-":
        not_after = int(not_after[:5])
    else:
        not_after = int(not_after[:4])# Replace em dash and +, handle full string
    return not_before, not_after



In [32]:
parse_dates("—0299-01/—0200-12")

(-299, -200)

In [33]:
oga_metadata["not_before"], oga_metadata["not_after"] = zip(*oga_metadata["formatted_work_date"].apply(parse_dates))

In [34]:
oga_metadata.head(5)

,id,urn_cts,title_labels,title_from_print_edition,print_edition,author,estimated_work_date,is_temporary_work_date,date_source,date_source_link,comment_on_chronology,formatted_work_date,date_label,not_before,not_after
0,1,ogl0001.ogl001,De Epistola Pinyti ad Dionysium,De Epistola Pinyti ad Dionysium,"Pinytus, Saint, Bishop of Knossos. Reliquiae S...","Pinytus, Saint, Bishop of Knossos",2nd Century AD (101-200),No,"Kritiko Agiologio, Holy Archibisophry of Crete",http://www.iak.gr/gr/ekklisia-kritis/kritiko_a...,Church of Crete Saints' Days Catalogue. The da...,+0101-01/+0200-12,p2_2,101,200
1,2,stoa0033a.tlg028,De mundo,De mundo,"pseudo-Aristotle, De mundo, Aristotelis Opera,...",Pseudo-Aristotle,250 BC-50BC,No,Pseudo-Aristotle: De Mundo (On the Cosmos). Ca...,https://www.cambridge.org/core/books/pseudoari...,Written between the middle of the 3rd and the ...,—0249-01/—0049-12,m3_2/m2_1/m2_2/m1_1,-249,-49
2,3,stoa0033a.tlg043,De spiritu,De spiritu,"pseudo-Aristotle, De spiritu, Aristotelis Oper...",Pseudo-Aristotle,275 BC-250 BC,No,Pseud-Aristotelian De Spiritu: A New Case agai...,,Follows Jaeger's proposed chronology.,—0274-01/—0249-12,m3_1,-274,-249
3,4,stoa0121.stoa001,Breviarium historiae romanae,Breviarium historiae romanae,Eutropius. Breviarium historiae romanae. Droys...,Eutropius,364 AD-378 AD,No,"Eutropius, Livius.org, 2020",https://www.livius.org/articles/person/eutropius/,Proposes an exact date of 369 AD.,+0364-01/+0378-12,p4_2,364,378
4,5,stoa0146d.stoa001,Acta Archelai,Acta Archelai,"Hegemonius. Acta Archelai. Beeson, Charles Hen...",Hegemonius,280 AD - 350 AD,No,"Archelaos, Wikisource",https://de.wikisource.org/wiki/RE:Archelaos_40,Based on the fact that he was bishop around 28...,+0280-01/+0350-12,p4_1,280,350


In [35]:
for col in oga_metadata.columns:
    oga_metadata.rename(columns={col : "oga_" + col}, inplace=True)
oga_metadata.head(5)

,oga_id,oga_urn_cts,oga_title_labels,oga_title_from_print_edition,oga_print_edition,oga_author,oga_estimated_work_date,oga_is_temporary_work_date,oga_date_source,oga_date_source_link,oga_comment_on_chronology,oga_formatted_work_date,oga_date_label,oga_not_before,oga_not_after
0,1,ogl0001.ogl001,De Epistola Pinyti ad Dionysium,De Epistola Pinyti ad Dionysium,"Pinytus, Saint, Bishop of Knossos. Reliquiae S...","Pinytus, Saint, Bishop of Knossos",2nd Century AD (101-200),No,"Kritiko Agiologio, Holy Archibisophry of Crete",http://www.iak.gr/gr/ekklisia-kritis/kritiko_a...,Church of Crete Saints' Days Catalogue. The da...,+0101-01/+0200-12,p2_2,101,200
1,2,stoa0033a.tlg028,De mundo,De mundo,"pseudo-Aristotle, De mundo, Aristotelis Opera,...",Pseudo-Aristotle,250 BC-50BC,No,Pseudo-Aristotle: De Mundo (On the Cosmos). Ca...,https://www.cambridge.org/core/books/pseudoari...,Written between the middle of the 3rd and the ...,—0249-01/—0049-12,m3_2/m2_1/m2_2/m1_1,-249,-49
2,3,stoa0033a.tlg043,De spiritu,De spiritu,"pseudo-Aristotle, De spiritu, Aristotelis Oper...",Pseudo-Aristotle,275 BC-250 BC,No,Pseud-Aristotelian De Spiritu: A New Case agai...,,Follows Jaeger's proposed chronology.,—0274-01/—0249-12,m3_1,-274,-249
3,4,stoa0121.stoa001,Breviarium historiae romanae,Breviarium historiae romanae,Eutropius. Breviarium historiae romanae. Droys...,Eutropius,364 AD-378 AD,No,"Eutropius, Livius.org, 2020",https://www.livius.org/articles/person/eutropius/,Proposes an exact date of 369 AD.,+0364-01/+0378-12,p4_2,364,378
4,5,stoa0146d.stoa001,Acta Archelai,Acta Archelai,"Hegemonius. Acta Archelai. Beeson, Charles Hen...",Hegemonius,280 AD - 350 AD,No,"Archelaos, Wikisource",https://de.wikisource.org/wiki/RE:Archelaos_40,Based on the fact that he was bishop around 28...,+0280-01/+0350-12,p4_1,280,350


In [36]:
oga_metadata["doc_id"] = oga_metadata["oga_urn_cts"]

In [37]:
sentences_path = "/srv/data/greek/oga_sentences_2025-08/"
os.listdir(sentences_path)[:10]

['tlg1264.tlg001.pickle',
 'tlg0007.tlg121.pickle',
 'pta0100.pta008.pickle',
 'tlg0527.tlg020.pickle',
 'tlg0540.tlg019.pickle',
 'tlg0026.tlg004.pickle',
 'tlg0018.tlg020.pickle',
 'tlg0540.tlg015.pickle',
 'tlg2042.tlg086.pickle',
 'pta0001.pta005.pickle']

In [38]:
doc_id = oga_metadata["doc_id"][0]
with open(os.path.join(sentences_path, doc_id + ".pickle"), "rb") as f:
    sentences_data = pickle.load(f)

In [39]:
sum([len(sent[3]) for sent in sentences_data])

81

In [40]:
def count_oga_tokens(doc_id):
    try:
        with open(os.path.join(sentences_path, doc_id + ".pickle"), "rb") as f:
            sentences_data = pickle.load(f)
        return sum([len(sent[3]) for sent in sentences_data])
    except:
        return None

In [41]:
oga_metadata["oga_tokencount"] = oga_metadata["doc_id"].apply(count_oga_tokens)

In [43]:
oga_metadata.columns

Index(['oga_id', 'oga_urn_cts', 'oga_title_labels',
       'oga_title_from_print_edition', 'oga_print_edition', 'oga_author',
       'oga_estimated_work_date', 'oga_is_temporary_work_date',
       'oga_date_source', 'oga_date_source_link', 'oga_comment_on_chronology',
       'oga_formatted_work_date', 'oga_date_label', 'oga_not_before',
       'oga_not_after', 'doc_id', 'oga_tokencount'],
      dtype='object')

In [44]:
oga_metadata = oga_metadata[['oga_id', 'oga_title_labels', 'oga_title_from_print_edition', 'oga_print_edition', 'oga_author', 'oga_date_source', 'oga_date_source_link', 'oga_comment_on_chronology', 'oga_formatted_work_date', 'oga_not_before', 'oga_not_after', 'doc_id', 'oga_tokencount']]

## Exprecce metadata

In [42]:
exprecce_metadata = pd.read_csv("../data/sources/exprecce/exprecce_metadata_v2.csv", sep=",")

In [43]:
exprecce_metadata.columns

Index(['author_id', 'doc_id', 'grela_id', 'author', 'title', 'raw_date',
       'not_before', 'not_after', 'lagt_genre', 'edition', 'edition_link',
       'lagt_provenience', 'textsource', 'grela_source', 'EXPRECCE?'],
      dtype='object')

In [44]:
def count_exprecce_tokens(doc_id):
    try:
        with open(os.path.join(exprecce_sentences_dir, doc_id + ".pickle"), "rb") as f:
            sentences_data = pickle.load(f)
        return sum([len(sent[3]) for sent in sentences_data])
    except:
        return None
exprecce_metadata["exprecce_tokencount"] = exprecce_metadata["doc_id"].apply(count_exprecce_tokens)

In [45]:
exprecce_metadata.head(5)

,author_id,doc_id,grela_id,author,title,raw_date,not_before,not_after,lagt_genre,edition,edition_link,lagt_provenience,textsource,grela_source,EXPRECCE?,exprecce_tokencount
0,tlg0304,tlg0304.tlg001,lagt_tlg0304.tlg001,NaN,Acta et martyrium Apollonii,A.D. 2/4,101,400.0,Hagiogr.,"Rudolf Knopf, Ausgewählte Märtyrerakten, 3. ne...",https://archive.org/details/ausgewahltemarty00...,christian,exprecce,lagt,True,2142
1,tlg0384,tlg0384.tlg002,lagt_tlg0384.tlg002,NaN,Acta Justini et septem sodalium (recensio B),A.D. 2/3,101,300.0,Hagiogr.,"Rudolf Knopf, Ausgewählte Märtyrerakten, 3. ne...",https://archive.org/details/ausgewahltemarty00...,christian,exprecce,lagt,True,956
2,tlg0388,tlg0388.tlg002,lagt_tlg0388.tlg002,NaN,Martyrium Pauli,A.D. 2,101,200.0,"Apocryph., Hagiogr.","R.A. Lipsius & Bonnet, M., Acta apostolorum ap...",https://archive.org/details/ntapoc-acta-aposto...,christian,exprecce,lagt,True,1424
3,tlg0388,tlg0388.tlg004,lagt_tlg0388.tlg004,NaN,Acta Pauli et Theclae,A.D. 2,101,200.0,"Apocryph., Hagiogr.","R.A. Lipsius & Bonnet, M., Acta apostolorum ap...",https://archive.org/details/ntapoc-acta-aposto...,christian,exprecce,lagt,True,4116
4,tlg0389,tlg0389.tlg001,lagt_tlg0389.tlg001,NaN,Martyrium Petri,A.D. 2,101,200.0,"Apocryph., Hagiogr.","R.A. Lipsius & Bonnet, M., Acta apostolorum ap...",https://archive.org/details/ntapoc-acta-aposto...,christian,exprecce,lagt,True,2903


In [84]:
for col in ["author", "title", "not_before", "not_after", "edition", "edition_link"]:
    exprecce_metadata.rename(columns={col : "exprecce_" + col}, inplace=True)

## Merge metadata

In [85]:
from functools import reduce

In [86]:
#LAGT3_metadata['LAGT3?'] = True
LAGT41_metadata['LAGT4-1?'] = True
glaux_metadata['GLAUX?'] = True
oga_metadata['OGA?'] = True
exprecce_metadata['EXPRECCE?'] = True

dataframes = [LAGT41_metadata, glaux_metadata, oga_metadata, exprecce_metadata]

In [87]:
gr_metadata_merged = reduce(lambda left, right: pd.merge(left, right, on="doc_id", how="outer"), dataframes)

In [88]:
presence_columns = ["LAGT4-1?", "GLAUX?", "OGA?", "EXPRECCE?"]
gr_metadata_merged[presence_columns] = gr_metadata_merged[presence_columns].fillna(False).astype(bool)

/tmp/ipykernel_3492435/336539419.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gr_metadata_merged[presence_columns] = gr_metadata_merged[presence_columns].fillna(False).astype(bool)


In [89]:
gr_metadata_merged.columns

Index(['doc_id', 'lagt4-1_author', 'lagt4-1_title', 'lagt4-1_not_before',
       'lagt4-1_not_after', 'lagt4-1_tlg_epithet', 'lagt4-1_source',
       'lagt4-1_genre', 'lagt4-1_provenience', 'lagt4-1_tokencount',
       'LAGT4-1?', 'glaux_GLAUX_TEXT_ID', 'glaux_TLG', 'glaux_STARTDATE',
       'glaux_ENDDATE', 'glaux_AUTHOR_STANDARD', 'glaux_TITLE_STANDARD',
       'glaux_GENRE_STANDARD', 'glaux_DIALECT', 'glaux_SOURCE',
       'glaux_SOURCE_LICENSE', 'glaux_SOURCE_FORMAT', 'glaux_TOKENS',
       'glaux_TM_TEXT', 'glaux_tlg_id', 'GLAUX?', 'oga_id', 'oga_urn_cts',
       'oga_title_labels', 'oga_title_from_print_edition', 'oga_print_edition',
       'oga_author', 'oga_estimated_work_date', 'oga_is_temporary_work_date',
       'oga_date_source', 'oga_date_source_link', 'oga_comment_on_chronology',
       'oga_formatted_work_date', 'oga_date_label', 'oga_not_before',
       'oga_not_after', 'oga_tokencount', 'OGA?', 'author_id', 'grela_id',
       'exprecce_author', 'exprecce_title', 'raw_d

In [90]:
gr_metadata_merged = gr_metadata_merged[[
    'doc_id',
    #'LAGT3?',
    'LAGT4-1?',
    'GLAUX?',
    'OGA?',
    'EXPRECCE?',
    'lagt4-1_author',
    'glaux_AUTHOR_STANDARD',
    'oga_author',
    'lagt4-1_title',
    'glaux_TITLE_STANDARD',
    'oga_title_labels',
    'oga_title_from_print_edition',
    'lagt4-1_not_before',
    'lagt4-1_not_after',
    'glaux_STARTDATE',
    'glaux_ENDDATE',
    'oga_not_before',
    'oga_not_after',
    'oga_date_source',
    'oga_date_source_link',
    'oga_comment_on_chronology',
    'oga_formatted_work_date',
    #'lagt3_tokencount',
    'lagt4-1_tokencount',
    'glaux_TOKENS',
    'oga_tokencount',
    'lagt4-1_source',
    'lagt4-1_tlg_epithet',
    'lagt4-1_genre',
    'lagt4-1_provenience',
    'glaux_GENRE_STANDARD',
    'glaux_DIALECT',
    'glaux_GLAUX_TEXT_ID',
    'glaux_TLG',
    'glaux_SOURCE',
    'glaux_SOURCE_FORMAT',
    'glaux_TM_TEXT',
    'exprecce_author',
    'exprecce_title',
    'exprecce_not_after',
    'exprecce_not_before',
    'exprecce_edition',
    'exprecce_edition_link',
    'oga_id',
    'oga_print_edition',
]]

In [91]:
gr_metadata_merged.sample(10)

,doc_id,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,glaux_AUTHOR_STANDARD,oga_author,lagt4-1_title,glaux_TITLE_STANDARD,oga_title_labels,oga_title_from_print_edition,lagt4-1_not_before,lagt4-1_not_after,glaux_STARTDATE,glaux_ENDDATE,oga_not_before,oga_not_after,oga_date_source,oga_date_source_link,oga_comment_on_chronology,oga_formatted_work_date,lagt4-1_tokencount,glaux_TOKENS,oga_tokencount,lagt4-1_source,lagt4-1_tlg_epithet,lagt4-1_genre,lagt4-1_provenience,glaux_GENRE_STANDARD,glaux_DIALECT,glaux_GLAUX_TEXT_ID,glaux_TLG,glaux_SOURCE,glaux_SOURCE_FORMAT,glaux_TM_TEXT,exprecce_author,exprecce_title,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition
2,pta0001.pta001,False,False,True,False,NaN,NaN,Severian of Gabala,NaN,NaN,De fide et lege naturae,Severianus Gabalensis: De fide et lege naturae...,NaN,NaN,NaN,NaN,400.0,409.0,,,temporary date of the author life,+0400-01/+0409-12,NaN,NaN,4495.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1694,"Vatikan, Biblioteca Apostolica Vaticana, graec..."
1504,tlg1193.tlg001,True,False,True,False,Ariston of Chios,NaN,Ariston of Chios,Testimonia et Fragmenta,NaN,Testimonia et Fragmenta,Testimonia et Fragmenta,-300.0,-201.0,NaN,NaN,-299.0,-200.0,"Britannica, T. Editors of Encyclopaedia (2024,...",https://www.britannica.com/biography/Ariston-o...,Based on the approximate time of Ariston's flo...,—0299-01/—0200-12,4098.0,NaN,3376.0,glaux1,[Philosophici/-ae],[],pagan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,793,"Ariston. Stoicorum veterum fragmenta, Vol. 1. ..."
729,tlg0062.tlg006,True,True,True,False,Lucian,Lucianus,Lucianus Samosatenus,Μυίας Ἐγκώμιον,Muscae encomium,Muscae encomium,The Fly,101.0,200.0,101.0,200.0,140.0,149.0,Lucian of Samosata project,http://lucianofsamosata.info/wiki/doku.php?id=...,Written during the decade of the 140s AD accor...,+0140-01/+0149-12,1381.0,1346.0,1341.0,glaux1,[Sophistae],[],pagan,Oratory,Attic/Koine,1550.0,0062-006,Perseus,XML,6362,NaN,NaN,NaN,NaN,NaN,NaN,452,"Lucian of Samosata, The Fly, Lucian vol. 1, Ha..."
46,pta0001.pta047,False,False,True,False,NaN,NaN,Severian of Gabala,NaN,NaN,In Iob sermo 3,Severianus Gabalensis: In Iob sermo 3,NaN,NaN,NaN,NaN,400.0,409.0,,,temporary date of the author life,+0400-01/+0409-12,NaN,NaN,4329.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1738,"Jean-Paul Migne (ed.), Patrologia graeca. 56. ..."
454,tlg0018.tlg022,True,True,True,False,Philo Judaeus,Philo Judaeus,Philo Judaeus,De vita Mosis (lib. i‑ii),De vita Mosis,De Vita Mosis (Lib. I-II),De Vita Mosis (Lib. I-II),-100.0,100.0,-100.0,100.0,40.0,49.0,"IEHOFF, M. R. (2018). Philo of Alexandria: An ...",https://doi.org/10.2307/j.ctt1z27jf9,Niehoff considers that Philo wrote his philoso...,+0040-01/+0049-12,36574.0,35631.0,35334.0,glaux1,[Philosophici/-ae],[],jewish,Biography,Koine,895.0,0018-022,https://el.wikisource.org,TXT,11191,NaN,NaN,NaN,NaN,NaN,NaN,306,"Philo Judaeus. Cohn, Leonard, editor. Opera qu..."
1951,tlg2200.tlg00507,True,False,True,False,Libanius,NaN,Libanius,Declamatio 7,NaN,Declamatio 7,Declamatio 7,301.0,400.0,NaN,NaN,350.0,393.0,"Weißenberger, M. (. (2006). Libanius. In Brill...",https://doi.org/10.1163/1574-9347_bnp_e703430,None,+0350-01/+0393-12,3252.0,NaN,2228.0,1Kgr,"[Rhetorici, Sophistae]",[],pagan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1447,"Libanius, Declamatio 7, Libanii Opera, Vol V, ..."
12,pta0001.pta011,False,False,True,False,NaN,NaN,Severian of Gabala,NaN,NaN,De serpente homilia,Severianus Gabalensis: De serpente homilia,NaN,NaN,NaN,NaN,400.0,409.0,,,temporary date of the author life,+0400-01/+0409-12,NaN,NaN,8909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1704,"Jean-Paul Migne (ed.), Patrologia graeca. 56. ..."
1699,tlg1804.tlg004,True,True,False,False,Ninus,Ninus,NaN,Ninus (fragmentum C),Ninus,NaN,NaN,-100.0,-1.0,-100.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,195.0,169.0,NaN,glaux1,None,[Narr. Fit.],pa

# Deduplicating

In [92]:
# Extract only the necessary columns
subset_df = gr_metadata_merged[["doc_id", "glaux_TLG"]]

# Group by 'doc_id' and aggregate 'glaux_TLG' values into a list
doc_glaux_tlgs_dict = gr_metadata_merged.groupby("doc_id")["glaux_TLG"].apply(list).to_dict()
doc_glaux_tokens_dict = gr_metadata_merged.groupby("doc_id")["glaux_TOKENS"].apply(np.sum).to_dict()

In [93]:
doc_glaux_tlgs_dict["tlg0007.tlg051"] # both texts together

['0007-051a', '0007-051b']

In [94]:
doc_glaux_tokens_dict["tlg0007.tlg051"] # sums of tokens from all components

15147.0

In [95]:
gr_metadata_merged["glaux_tlg_ids"] = gr_metadata_merged["doc_id"].apply(lambda x: doc_glaux_tlgs_dict[x])
gr_metadata_merged["glaux_tokens_sum"] = gr_metadata_merged["doc_id"].apply(lambda x: doc_glaux_tokens_dict[x])

In [96]:
# Get duplicate rows based on the "doc_id" column
duplicates = gr_metadata_merged[
    gr_metadata_merged.duplicated(subset=["doc_id"], keep=False)]
# View the duplicated rows
duplicates

,doc_id,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,glaux_AUTHOR_STANDARD,oga_author,lagt4-1_title,glaux_TITLE_STANDARD,oga_title_labels,oga_title_from_print_edition,lagt4-1_not_before,lagt4-1_not_after,glaux_STARTDATE,glaux_ENDDATE,oga_not_before,oga_not_after,oga_date_source,oga_date_source_link,oga_comment_on_chronology,oga_formatted_work_date,lagt4-1_tokencount,glaux_TOKENS,oga_tokencount,lagt4-1_source,lagt4-1_tlg_epithet,lagt4-1_genre,lagt4-1_provenience,glaux_GENRE_STANDARD,glaux_DIALECT,glaux_GLAUX_TEXT_ID,glaux_TLG,glaux_SOURCE,glaux_SOURCE_FORMAT,glaux_TM_TEXT,exprecce_author,exprecce_title,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition,glaux_tlg_ids,glaux_tokens_sum
179,tlg0007.tlg051,True,True,True,False,Plutarch,Plutarchus,Plutarch,Agis and Cleomenes,Agis et Cleomenes,Agis and Cleomenes,Agis and Cleomenes,1.0,200.0,1.0,200.0,96.0,116.0,"C.P. Jones, Towards a chronology of Plutarch's...",https://www.cambridge.org/core/journals/journa...,Jones believes that the mention of Quintus Sos...,+0096-01/+0116-12,15317.0,5068.0,15036.0,glaux1,"[Biographi, Philosophici/-ae]",[],pagan,Biography,Attic/Koine,1104.0,0007-051a,Perseus,XML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81,"Plutarch. Plutarch's Lives, Vol. X. Perrin, Be...","[0007-051a, 0007-051b]",15147.0
180,tlg0007.tlg051,True,True,True,False,Plutarch,Plutarchus,Plutarch,Agis and Cleomenes,Agis et Cleomenes,Agis and Cleomenes,Agis and Cleomenes,1.0,200.0,1.0,200.0,96.0,116.0,"C.P. Jones, Towards a chronology of Plutarch's...",https://www.cambridge.org/core/journals/journa...,Jones believes that the mention of Quintus Sos...,+0096-01/+0116-12,15317.0,10079.0,15036.0,glaux1,"[Biographi, Philosophici/-ae]",[],pagan,Biography,Attic/Koine,1105.0,0007-051b,Perseus,XML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81,"Plutarch. Plutarch's Lives, Vol. X. Perrin, Be...","[0007-051a, 0007-051b]",15147.0
181,tlg0007.tlg052,True,True,True,False,Plutarch,Plutarchus,Plutarch,Tiberius and Caius Gracchus,Tiberius et Gaius Gracchus,Tiberius and Caius Gracchus,Tiberius and Caius Gracchus,1.0,200.0,1.0,200.0,96.0,116.0,"C.P. Jones, Towards a chronology of Plutarch's...",https://www.cambridge.org/core/journals/journa...,Jones believes that the mention of Quintus Sos...,+0096-01/+0116-12,10398.0,5629.0,10245.0,glaux1,"[Biographi, Philosophici/-ae]",[],pagan,Biography,Attic/Koine,1106.0,0007-052a,Perseus,XML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,"Plutarch. Plutarch's Lives, Vol. X. Perrin, Be...","[0007-052a, 0007-052b]",10289.0
182,tlg0007.tlg052,True,True,True,False,Plutarch,Plutarchus,Plutarch,Tiberius and Caius Gracchus,Tiberius et Gaius Gracchus,Tiberius and Caius Gracchus,Tiberius and Caius Gracchus,1.0,200.0,1.0,200.0,96.0,116.0,"C.P. Jones, Towards a chronology of Plutarch's...",https://www.cambridge.org/core/journals/journa...,Jones believes that the mention of Quintus Sos...,+0096-01/+0116-12,10398.0,4660.0,10245.0,glaux1,"[Biographi, Philosophici/-ae]",[],pagan,Biography,Attic/Koine,1107.0,0007-052b,Perseus,XML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,"Plutarch. Plutarch's Lives, Vol. X. Perrin, Be...","[0007-052a, 0007-052b]",10289.0
212,tlg0007.tlg082,True,True,True,False,Plutarch,Plutarchus,Plutarch,Ἀποφθέγματα Λακωνικά,Apophthegmata Laconica,Apophthegmata Laconica,Sayings of Spartans,1.0,200.0,1.0,200.0,96.0,120.0,"C.P. Jones, Towards a chronology of Plutarch's...",https://www.cambridge.org/core/journals/journa...,Approximate for most philosophical works accor...,+0096-01/+0120-12,3261.0,16577.0,15910.0,glaux1,"[Biographi, Philosophici/-ae]",[],pagan,Philosophy,Attic/Koine,1137.0,0007-082,Perseus,XML,570 / 6523 / 6524,NaN,NaN,NaN,NaN,NaN,NaN,112,"Plutarch. Moralia, Vol. III. Babbitt, Frank Co...","[0007-082, 0007-082a, 0007-082b]",19795.0
213,tlg0007.tlg082,True,True,True,False,Plutarch,Plutarchus,Plutarch,Ἀποφθέγματα Λακωνικά,Apophthegmata Laconica,Apophthegmata Laconica,Sayings of Spartans,1.0,200.0,1.0,200.0,96.0,120.0,"C.P. Jones, Towards a chronology of Plutarc

In [97]:
# Deduplicate based on "doc_id" and keep the first occurrence
gr_metadata_merged = gr_metadata_merged.drop_duplicates(subset=["doc_id"], keep="first")

In [98]:
sum(gr_metadata_merged["GLAUX?"])

1408

In [99]:
sum(gr_metadata_merged["OGA?"])

1911

In [100]:
# in both GLAUX and OGA
sum((gr_metadata_merged["GLAUX?"] & gr_metadata_merged["OGA?"]))

1183

In [101]:
# only in GLAUX
sum((gr_metadata_merged["GLAUX?"] & ~gr_metadata_merged["OGA?"]))

225

In [102]:
# only in OGA
sum((~gr_metadata_merged["GLAUX?"] & gr_metadata_merged["OGA?"]))

728

In [103]:
sum((gr_metadata_merged["LAGT4-1?"] & ~gr_metadata_merged["OGA?"] & ~gr_metadata_merged["GLAUX?"]))

24

In [104]:
def load_from_sentences(row):
    doc_id = row["doc_id"]
    if row["OGA?"]:
        filepath = os.path.join(path, doc_id+".pickle")

    with open(filepath, "r") as f:
        sents_data = pickle.load(f)


In [105]:

gr_metadata_merged[gr_metadata_merged["LAGT4-1?"] & ~gr_metadata_merged["OGA?"] & ~gr_metadata_merged["GLAUX?"]]

,doc_id,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,glaux_AUTHOR_STANDARD,oga_author,lagt4-1_title,glaux_TITLE_STANDARD,oga_title_labels,oga_title_from_print_edition,lagt4-1_not_before,lagt4-1_not_after,glaux_STARTDATE,glaux_ENDDATE,oga_not_before,oga_not_after,oga_date_source,oga_date_source_link,oga_comment_on_chronology,oga_formatted_work_date,lagt4-1_tokencount,glaux_TOKENS,oga_tokencount,lagt4-1_source,lagt4-1_tlg_epithet,lagt4-1_genre,lagt4-1_provenience,glaux_GENRE_STANDARD,glaux_DIALECT,glaux_GLAUX_TEXT_ID,glaux_TLG,glaux_SOURCE,glaux_SOURCE_FORMAT,glaux_TM_TEXT,exprecce_author,exprecce_title,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition,glaux_tlg_ids,glaux_tokens_sum
1068,tlg0304.tlg001,True,False,False,True,,NaN,NaN,Acta et martyrium Apollonii,NaN,NaN,NaN,101.0,400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2123.0,NaN,NaN,exprecce,None,[Hagiogr.],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acta et martyrium Apollonii,400.0,101.0,"Rudolf Knopf, Ausgewählte Märtyrerakten, 3. ne...",https://archive.org/details/ausgewahltemarty00...,NaN,NaN,[nan],0.0
1094,tlg0389.tlg001,True,False,False,True,,NaN,NaN,Martyrdom of Peter,NaN,NaN,NaN,101.0,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2834.0,NaN,NaN,exprecce,None,"[Apocryph., Hagiogr.]",christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Martyrium Petri,200.0,101.0,"R.A. Lipsius & Bonnet, M., Acta apostolorum ap...",https://archive.org/details/ntapoc-acta-aposto...,NaN,NaN,[nan],0.0
1095,tlg0390.tlg001,True,False,False,True,,NaN,NaN,"Martyrium sanctorum Carpi, Papyli et Agathonicae",NaN,NaN,NaN,101.0,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1079.0,NaN,NaN,exprecce,None,[Hagiogr.],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Martyrium sanctorum Carpi, Papyli et Agathonicae",200.0,101.0,"Rudolf Knopf, Ausgewählte Märtyrerakten, 3. ne...",https://archive.org/details/ausgewahltemarty00...,NaN,NaN,[nan],0.0
1331,tlg0593.tlg003,True,False,False,False,Gorgias,NaN,NaN,Fragmenta (Fragment 11a),NaN,NaN,NaN,-500.0,-301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3062.0,NaN,NaN,glaux1,None,[Test.],pagan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan],0.0
1449,tlg0653.tlg003,True,False,False,False,Aratus Soleus,NaN,NaN,Epigrammata,NaN,NaN,NaN,-400.0,-201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,glaux1,None,[Epigr.],pagan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan],0.0
1498,tlg1157.tlg003,True,False,False,False,,NaN,NaN,Apocalypsis Esdrae,NaN,NaN,NaN,1.0,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2992.0,NaN,NaN,glaux1,None,[],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan],0.0
1569,tlg1352.tlg001,True,False,False,False,,NaN,NaN,The Letter of the Churches of Vienne and Lyons,NaN,NaN,NaN,101.0,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4161.0,NaN,NaN,exprecce,None,[Hagiogr.],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan],0.0
1698,tlg1804.tlg003,True,False,False,False,Ninus,NaN,NaN,Fragmenta A-B (P. Berol. 6926),NaN,NaN,NaN,-100.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1344.0,NaN,NaN,glaux1,None,[Narr. Fict.],pagan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan],0.0
1733,tlg2005.tlg001,True,False,False,True,,NaN,NaN,Martyrium Pionii presbyteri et sodalium,NaN,NaN,NaN,301.0,400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4827.0,NaN,NaN,exprecce,None,[Hagiogr.],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Martyrium Pionii presbyteri et sodalium,400.0,301.0,"Rudolf Knopf, Ausgewählte Märtyrerakten, 3. ne...",https://archive.org/details/ausgewahltemarty00...,NaN,NaN,[nan],0.0
1735,tlg2008.tlg001,True,False,False,True,,NaN,NaN,Martyrium Cononis,NaN,NaN,NaN,401.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1053.0,NaN,NaN,exprecce,None,[Hagiogr.],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Martyrium Cononis,NaN,401.0,"Rudolf Knopf, Ausgewählte Märtyrerakten, 3. ne...",https://archive.org/details/ausgewahltemarty00...,NaN,NaN,[nan],0.0


In [106]:
# only in OGA
sum((gr_metadata_merged["LAGT4-1?"] & ~gr_metadata_merged["OGA?"] & ~gr_metadata_merged["GLAUX?"]))

24

In [107]:
gr_metadata_merged["not_before"] = gr_metadata_merged["oga_not_before"].combine_first(gr_metadata_merged["lagt4-1_not_before"].combine_first(gr_metadata_merged["exprecce_not_before"]))
gr_metadata_merged["not_after"] = gr_metadata_merged["oga_not_after"].combine_first(gr_metadata_merged["lagt4-1_not_after"].combine_first(gr_metadata_merged["exprecce_not_after"]))

In [108]:
gr_metadata_merged["author"] = (
    gr_metadata_merged["exprecce_author"]
    .combine_first(gr_metadata_merged["lagt4-1_author"])
    .combine_first(gr_metadata_merged["oga_author"])
    .combine_first(gr_metadata_merged["glaux_AUTHOR_STANDARD"])
)

gr_metadata_merged["title"] = (
    gr_metadata_merged["exprecce_title"]
    .combine_first(gr_metadata_merged["lagt4-1_title"])
    .combine_first(gr_metadata_merged["oga_title_labels"])
    .combine_first(gr_metadata_merged["glaux_TITLE_STANDARD"])
)

In [122]:
gr_metadata_merged["lagt_tlg_epithet"] = gr_metadata_merged["lagt4-1_tlg_epithet"]
gr_metadata_merged["lagt_provenience"] = gr_metadata_merged["lagt4-1_provenience"]
gr_metadata_merged["lagt_tlg_epithet"] = gr_metadata_merged["lagt4-1_tlg_epithet"]
gr_metadata_merged["lagt_genre"] = gr_metadata_merged["lagt4-1_genre"]

In [124]:
gr_metadata_merged = gr_metadata_merged[['doc_id',  'author', 'title', 'not_before',
       'not_after', 'LAGT4-1?', 'GLAUX?', 'OGA?', 'EXPRECCE?', 'lagt4-1_author',
       'glaux_AUTHOR_STANDARD', 'oga_author', 'lagt4-1_title', 'lagt_genre', 'lagt_provenience', 'lagt_tlg_epithet',
       'glaux_TITLE_STANDARD', 'oga_title_labels',
       'oga_title_from_print_edition', 'lagt4-1_not_before',
       'lagt4-1_not_after', 'glaux_STARTDATE', 'glaux_ENDDATE',
       'oga_not_before', 'oga_not_after', 'oga_date_source',
       'oga_date_source_link', 'oga_comment_on_chronology',
       'oga_formatted_work_date', 'lagt4-1_tokencount', 'glaux_TOKENS',
       'oga_tokencount', 'lagt4-1_source', 'lagt4-1_tlg_epithet',
       'lagt4-1_genre', 'lagt4-1_provenience', 'glaux_GENRE_STANDARD',
       'glaux_DIALECT', 'glaux_GLAUX_TEXT_ID', 'glaux_TLG', 'glaux_SOURCE',
       'glaux_SOURCE_FORMAT', 'glaux_TM_TEXT', 'exprecce_author',
       'exprecce_title', 'exprecce_not_after', 'exprecce_not_before',
       'exprecce_edition', 'exprecce_edition_link', 'oga_id',
       'oga_print_edition', 'glaux_tlg_ids', 'glaux_tokens_sum']]

In [125]:
set_with_dataframe(lagt_metadata_gs.add_worksheet("gr_metadata_merged_2025-11-11", 1, 1), gr_metadata_merged)

In [126]:
gr_metadata_merged.to_csv("../data/gr_metadata_merged.csv", index=False)

In [129]:
test = pd.read_csv("https://raw.githubusercontent.com/sdam-au/LAGT/refs/heads/master/data/gr_metadata_merged.csv")

In [130]:
test.head(5)

,doc_id,author,title,not_before,not_after,LAGT4-1?,GLAUX?,OGA?,EXPRECCE?,lagt4-1_author,glaux_AUTHOR_STANDARD,oga_author,lagt4-1_title,lagt_tlg_epithet,lagt_provenience,lagt_tlg_epithet.1,glaux_TITLE_STANDARD,oga_title_labels,oga_title_from_print_edition,lagt4-1_not_before,lagt4-1_not_after,glaux_STARTDATE,glaux_ENDDATE,oga_not_before,oga_not_after,oga_date_source,oga_date_source_link,oga_comment_on_chronology,oga_formatted_work_date,lagt4-1_tokencount,glaux_TOKENS,oga_tokencount,lagt4-1_source,lagt4-1_tlg_epithet,lagt4-1_genre,lagt4-1_provenience,glaux_GENRE_STANDARD,glaux_DIALECT,glaux_GLAUX_TEXT_ID,glaux_TLG,glaux_SOURCE,glaux_SOURCE_FORMAT,glaux_TM_TEXT,exprecce_author,exprecce_title,exprecce_not_after,exprecce_not_before,exprecce_edition,exprecce_edition_link,oga_id,oga_print_edition,glaux_tlg_ids,glaux_tokens_sum
0,ggm0001.ggm001,Anonymous,Anametresis Pontou,1.0,400.0,False,False,True,False,NaN,NaN,Anonymous,NaN,NaN,NaN,NaN,NaN,Anametresis Pontou,Anametresis Pontou,NaN,NaN,NaN,NaN,1.0,400.0,"Kessler, K. (., & Talbert, R. (. H. N. (2006)....",https://doi.org/10.1163/1574-9347_bnp_e421860,NaN,+0001-01/+0400-12,NaN,NaN,422.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1607.0,"Anonymous. Geographi graeci minores, Volume 1....",[nan],0.0
1,ogl0001.ogl001,Pinytus,De Epistola Pinyti ad Dionysium,101.0,200.0,True,False,True,False,Pinytus,NaN,"Pinytus, Saint, Bishop of Knossos",De Epistola Pinyti ad Dionysium,[],christian,[],NaN,De Epistola Pinyti ad Dionysium,De Epistola Pinyti ad Dionysium,101.0,200.0,NaN,NaN,101.0,200.0,"Kritiko Agiologio, Holy Archibisophry of Crete",http://www.iak.gr/gr/ekklisia-kritis/kritiko_a...,Church of Crete Saints' Days Catalogue. The da...,+0101-01/+0200-12,108.0,NaN,81.0,glaux1,[],[],christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Pinytus, Saint, Bishop of Knossos. Reliquiae S...",[nan],0.0
2,pta0001.pta001,Severian of Gabala,De fide et lege naturae,400.0,409.0,False,False,True,False,NaN,NaN,Severian of Gabala,NaN,NaN,NaN,NaN,NaN,De fide et lege naturae,Severianus Gabalensis: De fide et lege naturae...,NaN,NaN,NaN,NaN,400.0,409.0,NaN,NaN,temporary date of the author life,+0400-01/+0409-12,NaN,NaN,4495.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1694.0,"Vatikan, Biblioteca Apostolica Vaticana, graec...",[nan],0.0
3,pta0001.pta002,Severian of Gabala,De paenitentia et compunctione,400.0,409.0,False,False,True,False,NaN,NaN,Severian of Gabala,NaN,NaN,NaN,NaN,NaN,De paenitentia et compunctione,Severianus Gabalensis: De paenitentia et compu...,NaN,NaN,NaN,NaN,400.0,409.0,NaN,NaN,temporary date of the author life,+0400-01/+0409-12,NaN,NaN,6987.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1695.0,"Jean-Paul Migne (ed.), Patrologia Graeca. Volu...",[nan],0.0
4,pta0001.pta003,Severian of Gabala,In ascensionem domini nostri Iesu Christi et i...,400.0,409.0,False,False,True,False,NaN,NaN,Severian of Gabala,NaN,NaN,NaN,NaN,NaN,In ascensionem domini nostri Iesu Christi et i...,Severianus Gabalensis: In ascensionem domini n...,NaN,NaN,NaN,NaN,400.0,409.0,NaN,NaN,temporary date of the author life,+0400-01/+0409-12,NaN,NaN,12070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1696.0,"Richard W. Bishop/Nathalie Rambault, Severian ...",[nan],0.0
